In [401]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, LineString, Point
from shapely.ops import split
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon

In [402]:
# Input Parameters
column_size = (5.0, 10.0)  # Column dimensions (x, y) in ft
column_eccentricity = (0.0, 0)  # Eccentricity of column from center (x, y) in ft
column_centroid = (8, 11)  # center of the column in ft
pile_cap_thickness = 5.5  # Pile cap thickness in ft
pile_embedment = 1.0  # Depth of pile embedment in pile cap (ft)
soil_depth_above = 2.0  # Soil depth above pile cap (ft)
soil_density = 120  # Soil density in pcf
concrete_density = 150  # Concrete density in pcf
pile_cap_shear_depth = 3.72  # Effective shear depth (ft)

In [403]:
# Pile Properties
pile_shape = "square"  # Either "square" or "circular"
pile_size = 2.0  # Pile size (ft)
max_pile_compression = 120  # Max pile resistance in compression (kips)
max_pile_tension = 150  # Max pile resistance in tension (kips)

In [404]:
# Column Loads (kips and kip-ft)
Fx, Fy, Fz = 39, 0.0, -4175  # Forces in kips
Mx, My = -8689, 1479  # Moments in kip-ft

In [405]:
# Pile Layout (x, y coordinates in ft) - measured from bottom-left corner
pile_layout = np.array([
    [2, 2], [8, 2], [14, 2],
    [2, 8], [8, 8], [14, 8],
    [2,14], [8,14], [14,14],
    [2,20], [8,20], [14,20]
])  # (ft)

In [406]:
# Pile Cap Shape (Vertices of pile cap)
pile_cap_vertices = np.array([
    [0, 0], [16, 0], [16, 22], [0, 22]
])  # Square pile cap

# Verify all piles are within the pile cap boundary
min_x, min_y = np.min(pile_cap_vertices, axis=0)
max_x, max_y = np.max(pile_cap_vertices, axis=0)

for i, (px, py) in enumerate(pile_layout):
    if not (min_x <= px <= max_x and min_y <= py <= max_y):
        raise ValueError(f"Pile {i+1} at ({px}, {py}) is outside the pile cap!")

In [407]:
# Compute Self-weight of Pile Cap and Soil
pile_cap_volume = 22 * 16 * pile_cap_thickness  # ft³
pile_cap_weight = (pile_cap_volume * concrete_density) / 1000  # kips

soil_volume = 22 * 16 * soil_depth_above  # ft³
soil_weight = (soil_volume * soil_density) / 1000  # kips

# Deduct pile embedment weight
pile_embedment_weight = (pile_embedment * pile_size**2 * len(pile_layout) * concrete_density) / 1000  # kips
total_weight = pile_cap_weight + soil_weight - pile_embedment_weight

In [408]:
# Compute centroid of pile locations
pile_centroid = np.mean(pile_layout, axis=0)

In [409]:
# Adjust loads for eccentricity
Mx_adj = Mx + Fz * column_eccentricity[1]  # Moment about x
My_adj = My - Fz * column_eccentricity[0]  # Moment about y

In [410]:
# Formulating Equilibrium Equations
num_piles = len(pile_layout)
A = np.zeros((3, num_piles))  # 3 equations (ΣFz=0, ΣMx=0, ΣMy=0)

# Right-hand side of the system of equations (Fix My sign)
b = np.array([-Fz + total_weight, -Mx_adj, My_adj])  # <-- FIX HERE

for i, (px, py) in enumerate(pile_layout):
    A[0, i] = 1  # ΣFz = 0
    A[1, i] = py - pile_centroid[1]  # ΣMx = 0 (Moment arm about x-axis)
    A[2, i] = px - pile_centroid[0]  # ΣMy = 0 (Moment arm about y-axis)

# Solve for reactions using least squares (in case of near-singular matrices)
reactions, _, _, _ = np.linalg.lstsq(A, b.reshape(-1, 1), rcond=None)

In [411]:
# Ensure piles are within capacity
for i, R in enumerate(reactions.flatten()):
    if R > max_pile_compression:
        print(f"WARNING: Pile {i+1} exceeds max compression ({R:.2f} > {max_pile_compression} kips)")
    if R < -max_pile_tension:
        print(f"WARNING: Pile {i+1} exceeds max tension ({abs(R):.2f} > {max_pile_tension} kips)")

In [412]:
# Compute One-Way Shear and Moment
critical_sections = {
    "Right": pile_centroid[0] + column_size[0] / 2 + pile_cap_shear_depth,
    "Top": pile_centroid[1] + column_size[1] / 2 + pile_cap_shear_depth
}

shear_moment_results = {}

# for location, section in critical_sections.items():
#     if location == "Right":
#         piles_contributing = [i for i in range(len(pile_layout)) if pile_layout[i, 0] >= section]
#         moment_arm = [pile_layout[i, 0] - section for i in piles_contributing]

#     elif location == "Top":
#         piles_contributing = [i for i in range(len(pile_layout)) if pile_layout[i, 1] >= section]
#         moment_arm = [pile_layout[i, 1] - section for i in piles_contributing]

#     # Calculate Shear and Moment
#     shear_force = sum(reactions[i, 0] for i in piles_contributing)

#     # Deduct weight contribution
#     affected_length = np.ptp(pile_cap_vertices[:, 1])  # Full height of pile cap
#     total_weight_contribution = (pile_cap_weight + soil_weight) * (affected_length / max_y)

#     # Apply weight deduction
#     shear_force -= total_weight_contribution

#     # Moment calculation
#     moment_about_section = sum(reactions[i, 0] * abs(moment_arm[idx]) for idx, i in enumerate(piles_contributing))

#     # Deduct weight moment contribution
#     centroid_distance = affected_length / 2
#     weight_moment = total_weight_contribution * centroid_distance

    # moment_about_section -= weight_moment

    # shear_moment_results[location] = {
    #     "Shear (kips)": shear_force,
    #     "Moment (kip-ft)": moment_about_section
    # }

In [413]:
# Display Results
results = pd.DataFrame({
    "Pile": np.arange(1, num_piles + 1),
    "X (ft)": pile_layout[:, 0],
    "Y (ft)": pile_layout[:, 1],
    "Reaction (kips)": reactions.flatten()
})

print("\nPile Reactions:\n", results)

# Print One-Way Shear and Moment
# print("\nOne-Way Shear and Moment at Critical Sections:")
# for loc, values in shear_moment_results.items():
#     print(f"{loc} of Column -> Shear: {values['Shear (kips)']:.2f} kips, Moment: {values['Moment (kip-ft)']:.2f} kip-ft")


Pile Reactions:
     Pile  X (ft)  Y (ft)  Reaction (kips)
0      1       2       2       202.927500
1      2       8       2       233.740000
2      3      14       2       264.552500
3      4       2       8       299.471944
4      5       8       8       330.284444
5      6      14       8       361.096944
6      7       2      14       396.016389
7      8       8      14       426.828889
8      9      14      14       457.641389
9     10       2      20       492.560833
10    11       8      20       523.373333
11    12      14      20       554.185833


In [414]:
#Function to split polygon by a line (used for one-way shear and moment)

def split_polygon_by_line(polygon_vertices, line_type, line_value, column_centroid=None):
    """
    Splits a polygon into two areas based on a line.

    Args:
    - polygon_vertices (np.ndarray): Vertices of the polygon [[x, y], ...].
    - line_type (str): 'regular' or 'vertical'.
    - line_value (tuple): (m, c) if regular or (x,) if vertical.
    - column_centroid (tuple): Not used in this version, but kept for future extension.

    Returns:
    - Tuple of two areas: (area_above_or_right, area_below_or_left)
    """
    # Create the main polygon
    polygon = Polygon(polygon_vertices)
    
    if not polygon.is_valid:
        raise ValueError("Input polygon is invalid! Check vertices order and if it's closed.")

    # Get bounds for extending the line beyond the polygon
    min_x, min_y, max_x, max_y = polygon.bounds
    extend = max(max_x - min_x, max_y - min_y) * 10  # Extend line far beyond bounds

    # Create the line geometry
    if line_type == 'regular':
        m, c = line_value
        # Pick x values far beyond polygon
        x1 = min_x - extend
        x2 = max_x + extend
        y1 = m * x1 + c
        y2 = m * x2 + c
        line = LineString([(x1, y1), (x2, y2)])

    elif line_type == 'vertical':
        x = line_value[0]
        y1 = min_y - extend
        y2 = max_y + extend
        line = LineString([(x, y1), (x, y2)])
    
    else:
        raise ValueError("line_type must be 'regular' or 'vertical'")

    # Split the polygon
    try:
        split_polygons = split(polygon, line)
    except Exception as e:
        print("Polygon could not be split:", e)
        return 0.0, 0.0

    # If not split, return the area as one side and zero on the other
    if len(split_polygons.geoms) < 2:
        print("Polygon was not split by the line.")
        return polygon.area, 0.0

    # Decide which side is "above/right" vs "below/left"
    areas = []
    for poly in split_polygons.geoms:
        centroid = poly.centroid
        x, y = centroid.x, centroid.y

        if line_type == 'regular':
            y_on_line = m * x + c
            if y > y_on_line:
                areas.append(('above', poly.area))
            else:
                areas.append(('below', poly.area))

        elif line_type == 'vertical':
            if x > line_value[0]:
                areas.append(('right', poly.area))
            else:
                areas.append(('left', poly.area))

    # Sum areas
    if line_type == 'regular':
        area_above = sum(a for s, a in areas if s == 'above')
        area_below = sum(a for s, a in areas if s == 'below')
        return area_above, area_below

    elif line_type == 'vertical':
        area_right = sum(a for s, a in areas if s == 'right')
        area_left = sum(a for s, a in areas if s == 'left')
        return area_right, area_left

In [415]:
# Example 1: Regular line (y = 0.5x + 5)
line_type = 'regular'
line_value = (0, 19.72)
areas = split_polygon_by_line(pile_cap_vertices, line_type, line_value)
print(f"Areas above/below the line y = 0.5x + 5: {areas}")

# Example 2: Vertical line (x = 8)
line_type = 'vertical'
line_value = (14.22,)
areas = split_polygon_by_line(pile_cap_vertices, line_type, line_value)
print(f"Areas right/left of the vertical line x = 8: {areas}")

Areas above/below the line y = 0.5x + 5: (36.48000000000002, 315.52)
Areas right/left of the vertical line x = 8: (39.15999999999998, 312.84000000000003)


In [416]:
#Function to know which piles are split by a line (for one-way shear and moment)

def analyze_pile_distribution(polygon_vertices, line_type, line_value, pile_layout, pile_size):
    from shapely.geometry import Polygon, LineString, box
    from shapely.ops import split

    pile_cap_polygon = Polygon(polygon_vertices)

    if not pile_cap_polygon.is_valid:
        raise ValueError("Invalid pile cap polygon.")

    # Create cutting line
    min_x, min_y, max_x, max_y = pile_cap_polygon.bounds
    extend = max(max_x - min_x, max_y - min_y) * 10

    if line_type == 'regular':
        m, c = line_value
        x1 = min_x - extend
        x2 = max_x + extend
        y1 = m * x1 + c
        y2 = m * x2 + c
        cutting_line = LineString([(x1, y1), (x2, y2)])
    elif line_type == 'vertical':
        x = line_value[0]
        y1 = min_y - extend
        y2 = max_y + extend
        cutting_line = LineString([(x, y1), (x, y2)])
    else:
        raise ValueError("line_type must be 'regular' or 'vertical'")

    # Initialize lists and areas
    piles_above = []
    piles_below = []
    intersected_piles = []
    area_above = 0.0
    area_below = 0.0
    intersected_area_above = 0.0
    intersected_area_below = 0.0

    half_size = pile_size / 2
    pile_area = pile_size ** 2

    for idx, (px, py) in enumerate(pile_layout):
        pile_box = box(px - half_size, py - half_size, px + half_size, py + half_size)

        # Check if pile intersects the cutting line
        if pile_box.crosses(cutting_line):
            intersected_piles.append(idx + 1)

            try:
                split_pile = split(pile_box, cutting_line)

                for piece in split_pile.geoms:
                    centroid = piece.centroid
                    cx, cy = centroid.x, centroid.y

                    if line_type == 'regular':
                        y_on_line = m * cx + c
                        if cy > y_on_line:
                            intersected_area_above += piece.area
                        else:
                            intersected_area_below += piece.area
                    elif line_type == 'vertical':
                        if cx > line_value[0]:
                            intersected_area_above += piece.area
                        else:
                            intersected_area_below += piece.area

            except Exception as e:
                print(f"Warning: Could not split pile {idx + 1}. Error: {e}")

        else:
            # Find relation of pile centroid to line, only if NOT intersecting
            if line_type == 'regular':
                y_on_line = m * px + c
                if py > y_on_line:
                    piles_above.append(idx + 1)
                    area_above += pile_area
                else:
                    piles_below.append(idx + 1)
                    area_below += pile_area
            elif line_type == 'vertical':
                if px > line_value[0]:
                    piles_above.append(idx + 1)
                    area_above += pile_area
                else:
                    piles_below.append(idx + 1)
                    area_below += pile_area

    # Final areas include the split areas from intersected piles
    total_area_above = area_above + intersected_area_above
    total_area_below = area_below + intersected_area_below

    return (
        total_area_above, total_area_below,
        piles_above, piles_below,
        intersected_area_above, intersected_area_below,
        intersected_piles
    )


In [417]:
# Example line: y = 0.5x + 5
line_type = 'regular'
line_value = (0, 19.72)

# Analyze pile areas
(
    area_above, area_below,
    piles_above, piles_below,
    intersected_area_above, intersected_area_below,
    intersected_piles
) = analyze_pile_distribution(
    pile_cap_vertices, line_type, line_value, pile_layout, pile_size
)

# Results
print(f"Total pile area above the line: {area_above:.2f} sq.ft.")
print(f"Total pile area below the line: {area_below:.2f} sq.ft.\n")

print(f"Piles above the line: {piles_above}")
print(f"Piles below the line: {piles_below}\n")

print(f"Intersected pile areas above: {intersected_area_above:.2f} sq.ft.")
print(f"Intersected pile areas below: {intersected_area_below:.2f} sq.ft.")
print(f"Intersected piles: {intersected_piles}")

Total pile area above the line: 7.68 sq.ft.
Total pile area below the line: 40.32 sq.ft.

Piles above the line: []
Piles below the line: [1, 2, 3, 4, 5, 6, 7, 8, 9]

Intersected pile areas above: 7.68 sq.ft.
Intersected pile areas below: 4.32 sq.ft.
Intersected piles: [10, 11, 12]


In [418]:
# Set line type and value for vertical line
line_type = 'vertical'
line_value = (10.5,)

# Run analysis
(
    area_right, area_left,
    piles_right, piles_left,
    intersected_area_right, intersected_area_left,
    intersected_piles
) = analyze_pile_distribution(
    pile_cap_vertices,
    line_type,
    line_value,
    pile_layout,
    pile_size
)

# Print Results
print(f"Total pile area right of the line: {area_right:.2f} sq.ft.")
print(f"Total pile area left of the line: {area_left:.2f} sq.ft.\n")

print(f"Piles right of the line: {piles_right}")
print(f"Piles left of the line: {piles_left}\n")

print(f"Intersected pile areas right: {intersected_area_right:.2f} sq.ft.")
print(f"Intersected pile areas left: {intersected_area_left:.2f} sq.ft.")
print(f"Intersected piles: {intersected_piles}")


Total pile area right of the line: 16.00 sq.ft.
Total pile area left of the line: 32.00 sq.ft.

Piles right of the line: [3, 6, 9, 12]
Piles left of the line: [1, 2, 4, 5, 7, 8, 10, 11]

Intersected pile areas right: 0.00 sq.ft.
Intersected pile areas left: 0.00 sq.ft.
Intersected piles: []


In [419]:
# Function to return weight of concrete and soil

def calculate_concrete_soil_weight(pile_cap_thickness, soil_depth_above, concrete_density, soil_density):
    """
    Calculates the weight of concrete and soil over a given area of pile cap.

    Parameters:
    - pile_cap_thickness: Thickness of the pile cap (ft)
    - soil_depth_above: Depth of soil above the pile cap (ft)
    - concrete_density: Density of concrete (pcf)
    - soil_density: Density of soil (pcf)

    Returns:
    - concrete_weight: Weight of concrete over the area (kips)
    - soil_weight: Weight of soil over the area (kips)
    """

    try:
        # Ask user for area input (ensure it's a float)
        area = float(input("Enter Area of pile cap (as split by section) in sq.ft.: "))

        # Calculate volumes (in cubic feet)
        concrete_volume = area * pile_cap_thickness
        soil_volume = area * soil_depth_above

        # Calculate weights (convert pcf to kips by dividing by 1000)
        concrete_weight = (concrete_volume * concrete_density) / 1000  # in kips
        soil_weight = (soil_volume * soil_density) / 1000  # in kips

        # print(f"\nConcrete Weight: {concrete_weight:.3f} kips")
        # print(f"Soil Weight: {soil_weight:.3f} kips\n")

        return concrete_weight, soil_weight

    except ValueError:
        print("Invalid input! Please enter a valid numeric area.")
        return None, None


In [420]:
# Example values for arguments:
pile_cap_thickness = 5.5  # feet
soil_depth_above = 2.0    # feet
concrete_density = 150    # pcf
soil_density = 120        # pcf

# Run the function
concrete_wt, soil_wt = calculate_concrete_soil_weight(
    pile_cap_thickness,
    soil_depth_above,
    concrete_density,
    soil_density
)

print(f"Returned concrete weight: {concrete_wt:.3f} kips")
print(f"Returned soil weight: {soil_wt:.3f} kips")


Enter Area of pile cap (as split by section) in sq.ft.:  45


Returned concrete weight: 37.125 kips
Returned soil weight: 10.800 kips


In [421]:
from shapely.geometry import Polygon, box

def analyze_shear_polygon_reactions(
    polygon_vertices,
    pile_layout,
    pile_size,
    column_centroid,
    column_size,
    shear_depth,
    pile_reactions
):
    """
    Analyze reactions outside the shear polygon around a column, and categorize piles.

    Parameters:
    - polygon_vertices: np.array of pile cap polygon [[x1,y1], [x2,y2], ...]
    - pile_layout: np.array of pile center coordinates [[x1,y1], ...]
    - pile_size: size of each square pile (ft)
    - column_centroid: (x, y) tuple for column center
    - column_size: (width, height) of column (ft)
    - shear_depth: shear depth to expand the critical polygon (ft)
    - pile_reactions: np.array of reactions at each pile (kip)

    Returns:
    - total_reaction_outside: Sum of reactions outside the shear polygon (kip)
    - shear_polygon_coords: Coordinates of the shear polygon (list)
    - shear_polygon_perimeter: Perimeter of the shear polygon (ft)
    - inside_piles: List of pile indices fully inside the shear polygon
    - outside_piles: List of pile indices fully outside the shear polygon
    - intersecting_piles: List of pile indices partially inside/outside the shear polygon
    """

    # Create pile cap polygon
    pile_cap_polygon = Polygon(polygon_vertices)

    if not pile_cap_polygon.is_valid:
        raise ValueError("Invalid pile cap polygon provided.")

    # Shear polygon dimensions (centered on column centroid)
    col_x, col_y = column_centroid
    col_width, col_height = column_size

    # Expand by shear depth / 2 on all sides
    shear_width = col_width + shear_depth
    shear_height = col_height + shear_depth

    # Coordinates of shear polygon BEFORE trimming
    shear_min_x = col_x - shear_width / 2
    shear_max_x = col_x + shear_width / 2
    shear_min_y = col_y - shear_height / 2
    shear_max_y = col_y + shear_height / 2

    # Create shear polygon box
    shear_polygon = box(shear_min_x, shear_min_y, shear_max_x, shear_max_y)

    # Trim to pile cap polygon (if it extends outside)
    shear_polygon_trimmed = shear_polygon.intersection(pile_cap_polygon)

    # Compute perimeter length of shear polygon
    shear_polygon_perimeter = shear_polygon_trimmed.length

    # Prepare totals
    total_reaction_outside = 0.0

    # Prepare lists for pile classifications
    inside_piles = []
    outside_piles = []
    intersecting_piles = []

    # Loop through each pile and check its relation to shear polygon
    half_size = pile_size / 2
    pile_area = pile_size ** 2

    for idx, (px, py) in enumerate(pile_layout):
        reaction = pile_reactions[idx]

        # Create pile box
        pile_box = box(px - half_size, py - half_size, px + half_size, py + half_size)

        # Fully inside shear polygon => no contribution to reaction outside
        if shear_polygon_trimmed.contains(pile_box):
            inside_piles.append(idx + 1)

        # Fully outside shear polygon => full reaction counts
        elif not shear_polygon_trimmed.intersects(pile_box):
            outside_piles.append(idx + 1)
            total_reaction_outside += reaction

        # Intersecting pile => partial reaction contribution
        else:
            intersecting_piles.append(idx + 1)
            
            # Get portion of pile box outside the shear polygon
            outside_piece = pile_box.difference(shear_polygon_trimmed)
            outside_area = outside_piece.area

            # Calculate reaction proportional to the area outside
            reaction_contribution = reaction * (outside_area / pile_area)
            total_reaction_outside += reaction_contribution

    # Get coordinates of trimmed shear polygon for output
    shear_polygon_coords = list(shear_polygon_trimmed.exterior.coords)

    # Return all results
    return (
        total_reaction_outside,
        shear_polygon_coords,
        shear_polygon_perimeter,
        inside_piles,
        outside_piles,
        intersecting_piles
    )


In [423]:
result = analyze_shear_polygon_reactions(
    polygon_vertices=pile_cap_vertices,
    pile_layout=pile_layout,
    pile_size=pile_size,
    column_centroid=column_centroid,
    column_size=column_size,
    shear_depth=shear_depth,
    pile_reactions=pile_reactions
)

(
    total_reaction_outside,
    shear_polygon_coords,
    shear_polygon_perimeter,
    inside_piles,
    outside_piles,
    intersecting_piles
) = result

print(f"Total reaction outside shear polygon: {total_reaction_outside:.2f} kip")
print(f"Shear polygon perimeter length: {shear_polygon_perimeter:.2f} ft")
print(f"Shear polygon coordinates: {shear_polygon_coords}")

print(f"Piles fully inside shear polygon: {inside_piles}")
print(f"Piles fully outside shear polygon: {outside_piles}")
print(f"Piles intersecting shear polygon: {intersecting_piles}")


Total reaction outside shear polygon: 4164.50 kip
Shear polygon perimeter length: 44.88 ft
Shear polygon coordinates: [(3.6399999999999997, 4.14), (3.6399999999999997, 17.86), (12.36, 17.86), (12.36, 4.14), (3.6399999999999997, 4.14)]
Piles fully inside shear polygon: []
Piles fully outside shear polygon: [1, 2, 3, 4, 6, 7, 9, 10, 11, 12]
Piles intersecting shear polygon: [5, 8]
